## Method2

In [1]:
from utils import *

#### Load a file

In [2]:
mesh = pymesh.load_mesh("data/two_spheres.ply")

In [ ]:
self_intersection_stats(mesh)

In [ ]:
visualize(mesh)

In [ ]:
visualize_intersection(mesh)

#### Get outer hull

In [3]:
outer_hull = pymesh.compute_outer_hull(mesh)

In [ ]:
self_intersection_stats(outer_hull)

#### Create a bounding box

In [39]:
local_min = np.array([ -0.90, -0.90, -0.90])
local_max = np.array([ 0.90,  0.10,  0.90])

In [40]:
submesh, face_mask = extract_submesh_by_bbox(outer_hull, local_min, local_max)

In [ ]:
visualize(submesh, filename="Submesh")

In [41]:
remaining_mesh = extract_remaining_mesh(outer_hull, face_mask)

In [ ]:
visualize(remaining_mesh, filename="remaining_mesh")

#### Repair (or Remesh) the submesh
- `pymesh.collapse_short_edges(mesh, abs_threshold, rel_threshold, preserve_feature)`<br>
Collapses all edges with length less than a user specified threshold.

- `pymesh.split_long_edges(mesh, max_edge_length)`<br>
Splits long edges into 2 or more shorter edges.

- `pymesh.remove_obtuse_triangles(mesh, max_angle, max_iterations)`<br>
Splits each obtuse triangle into 2 or more right or sharp triangles.

- `pymesh.remove_degenerated_triangles(mesh, num_iterations)` <br>
Degenerate triangles are triangles with collinear vertices (i.e., some vertices are lying in the same straight line). They have zero areas and their normals are undefined.

In [42]:
from numpy.linalg import norm

def method2(mesh, detail=2e-2):

    bbox_min, bbox_max = mesh.bbox
    diag_len = norm(bbox_max - bbox_min)
    target_len = diag_len * detail
    print("Target resolution: {} mm".format(target_len))

    count = 0
    mesh, __ = pymesh.remove_degenerated_triangles(mesh, 100)
    mesh, __ = pymesh.split_long_edges(mesh, target_len)
    num_vertices = mesh.num_vertices
    while True:
        mesh, __ = pymesh.collapse_short_edges(mesh, 1e-6) # Remove extremely small edges
        mesh, __ = pymesh.collapse_short_edges(mesh, target_len,
                                               preserve_feature=True)
        mesh, __ = pymesh.remove_obtuse_triangles(mesh, 150.0, 100)
        if mesh.num_vertices == num_vertices:
            break

        num_vertices = mesh.num_vertices
        print("#v: {}".format(num_vertices))
        count += 1
        if count > 10: break

    mesh = pymesh.resolve_self_intersection(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh = pymesh.compute_outer_hull(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh, __ = pymesh.remove_obtuse_triangles(mesh, 179.0, 5)
    mesh, __ = pymesh.remove_isolated_vertices(mesh)

    return mesh

In [43]:
repaired_submesh = method2(submesh)

Target resolution: 0.061370150332789634 mm
#v: 980
#v: 977


In [ ]:
visualize(repaired_submesh, filename="Repaired_submesh")

#### Align submesh
Since we have modified the submesh, we need to fix its boundary.

In [44]:
aligned_submesh = align_submesh_boundary(remaining_mesh, repaired_submesh)

In [ ]:
visualize(aligned_submesh, filename="Method2")

#### Merge two meshes

In [45]:
repaired_full = replace_submesh_in_original(remaining_mesh, aligned_submesh)

#### Final refinement

In [46]:
final = refinement(repaired_full)

In [47]:
self_intersection_stats(final)

+-----------------------------------+---------+
| Metric                            |   Value |
+===================================+=========+
| Number of vertices                |     948 |
+-----------------------------------+---------+
| Number of faces                   |    1892 |
+-----------------------------------+---------+
| Number of intersecting face pairs |       0 |
+-----------------------------------+---------+


In [48]:
visualize(final, filename="Final Outerhull")

Widget(value='<iframe src="http://localhost:37513/index.html?ui=P_0x7fac8cff6b30_2&reconnect=auto" class="pyvi…

None

In [49]:
evaluation(outer_hull, final)

+-----------------------------------+-----------+------------+
| Metric                            |    Before |      After |
+===================================+===========+============+
| Number of vertices                | 264       |  948       |
+-----------------------------------+-----------+------------+
| Number of faces                   | 524       | 1892       |
+-----------------------------------+-----------+------------+
| Number of intersecting face pairs |   0       |    0       |
+-----------------------------------+-----------+------------+
| Volume                            |   6.10774 |    6.10366 |
+-----------------------------------+-----------+------------+
| Area                              |  17.6163  |   17.6117  |
+-----------------------------------+-----------+------------+


In [50]:
evaluate_displacement(outer_hull, final)

Max Vertex Displacement: 0.2651650429449553
Mean Vertex Displacement: 0.07841682882135739


In [51]:
pymesh.save_mesh("final.ply", final, ascii=True)

In [31]:
x = pymesh.load_mesh("final.ply")